In [ ]:
from os import listdir
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
import pandas as pd

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
def preprocessing(text_file):
    tokenizer = nltk.RegexpTokenizer(r"\w+")    # remove punctuations
    tokens = tokenizer.tokenize(text_file)    # token the text

    tokens = [token.lower() for token in tokens]    # lower case
    tokens = [word for word in tokens if not word in stop_words]    # remove stop words

    stemmer = PorterStemmer()    #stemming tokens
    tokens = [stemmer.stem(word) for word in tokens]
    
    lemmatizer = WordNetLemmatizer()   #lemmatizing tokens
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    return tokens

# get a list of all files in the directory
def getFilesList(folder):
    dir_list = listdir(folder)    #subfolders
    files = {}
    for i in dir_list:
        path = folder+i+"/"
        diri = listdir(path)    #files per subfolder
        files[i] = []
        for j in diri:
            filepath = path+j
            files[i].append(filepath)    #storing full path of file
    return files

def files_read_preprocess(folderpath):
    flist = getFilesList(folderpath)
    alltextfromfiles = []
    textlabels = []
    for folder in flist:
        for file in flist[folder]:
            text = open(file,'r',encoding='ISO-8859-1').read()
            tokens = preprocessing(text)
            alltextfromfiles.append(tokens)
            textlabels.append(folder)    #corresponding label of text

    print(len(alltextfromfiles))
    print(len(textlabels))

    dataset = pd.DataFrame([alltextfromfiles,textlabels]).T
    dataset.to_pickle("q3_preprocessed")

In [ ]:
train_prior = Counter(y_train)
print(train_prior)

class_dict = {}
for i in range(X_train.shape[0]):
    if y_train[i] not in class_dict:
        class_dict[y_train[i]] = []
    class_dict[y_train[i]] += X_train[i]

uniq_labels = listdir(folderpath)
class_freq,class_count = termfrequency_tf(uniq_labels,class_dict)

unique_words = set()
for i in class_dict:
    unique_words = unique_words | set(class_dict[i])

unique_words_count = len(unique_words)
print(len(class_frequency))
print(class_count)

In [ ]:
def tficf_feature_select(x_train,y_train,top_k):
    corpus = []
    for i in class_dict:
        corpus += class_dict[i]

    # cf = {}
    # n = 0
    
    # for i in class_dict:
    #     for word in class_dict[i]:
    #         if word not in cf:
    #             cf[word] = {}
    #         cf[word].add(n)
    #     n += 1

    # for word in cf:
    #     cf[word] = len(cf[word])
    
    # tficf = {}
    # sizetrain = train.shape[0]

    # counter = Counter(corpus)
    # word_count = len(corpus)

    # for token in set(corpus):
    #     cf = 


def getCountFreqTopKFeatures(feature_list):
    class_frequency = {}
    class_count = {}
    counter = 0
    for i in labels:
        current_count = len(Counter(feature_list[i]))
        class_count[i] = current_count
        counter += current_count
        ll = Counter(feature_list[i])
        for j in ll:
            class_frequency[i, j] = ll[j]
        
    return class_frequency,class_count

def Naive_bayes(class_frequency,class_count,x_test,y_test):
    actual = []
    predicted = []

    for i in range(x_test.shape[0]):
        classes_word_prob = []
        actual.append(y_test[i])

        for labels in uniq_labels:
            word_prob = 0
            for word in x_test[i]:
                t1,t2 = 0,class_count[labels]
                if labels,word in class_frequency:
                    t1 = class_frequency[labels,word]
                    
                t3 = (t1+1)/(t2+unique_words_count)
                word_prob += np.log(t2)
            classes_word_prob.append(word_prob)
        predicted.append(uniq_labels[np.argmax(classes_word_prob)])

    return actual,predicted

In [ ]:
folderpath = "20_newsgroups/"
# dataset = files_read_preprocess(folderpath)
dataset = pd.read_pickle("q3_preprocessed")
# print(dataset)

y = dataset.pop(1)
X = dataset
splits = [0.2,0.3,0.5]

for splitsize in splits:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=splitsize, random_state=42)
    
    sorted_feature_list = tficf_feature_select(X_train,y_train,10)
    class_frequecy1,class_count1 = getCountFreqTopKFeatures(sorted_feature_list)
    actual,predicted = Naive_Bayes(class_frequecy1,class_count1)

    acc = accuracy_score(actual,predicted)
    print(acc)
    confusion = confusion_matrix(actual,predicted)
    print(confusion)

                                                      0                 1
0     [path, cantaloup, srv, c, cmu, edu, da, news, ...  rec.sport.hockey
1     [newsgroup, rec, sport, hockey, path, cantalou...  rec.sport.hockey
2     [path, cantaloup, srv, c, cmu, edu, da, news, ...  rec.sport.hockey
3     [newsgroup, rec, sport, hockey, path, cantalou...  rec.sport.hockey
4     [newsgroup, rec, sport, hockey, path, cantalou...  rec.sport.hockey
...                                                 ...               ...
4995  [xref, cantaloup, srv, c, cmu, edu, sci, med, ...           sci.med
4996  [path, cantaloup, srv, c, cmu, edu, rochest, u...           sci.med
4997  [xref, cantaloup, srv, c, cmu, edu, sci, med, ...           sci.med
4998  [path, cantaloup, srv, c, cmu, edu, crabappl, ...           sci.med
4999  [xref, cantaloup, srv, c, cmu, edu, sci, med, ...           sci.med

[5000 rows x 2 columns]


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5be6439c-c857-493e-9eef-529c4c49e2bf' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>